## Optimizing Supply Chain Management

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import RFE
from xgboost import XGBRegressor

In [2]:
# Load the data and display the first few rows of the dataset
data = pd.read_csv('/Users/sk/Desktop/AIchemyLab/2_ML/ML_Reg_FMCG_Supply_Chain/data/raw_data.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw_data.csv'

In [ ]:
# Basic information about the dataset
data.info()

The dataset contains 25,000 rows and 24 columns.

Here are some key details:
1. Data Types:
2 float64 columns, 
14 int64 columns, 
8 object columns

2. Columns with Missing Values:
workers_num: 24,010 non-null, 
wh_est_year: 13,119 non-null, 
approved_wh_govt_certificate: 24,092 non-null

In [ ]:
# Checking for missing values
data.isnull().sum()

Here are the missing values for each column in the dataset:
1. workers_num: 990 missing values
2. wh_est_year: 11,881 missing values
3. approved_wh_govt_certificate: 908 missing values

In [ ]:
# Checking for unique values
data.nunique()

In [ ]:
# Summary statistics of the dataset
data.describe().T

In [ ]:
# Set the overall theme and color palette
sns.set_theme(style="whitegrid")
sns.set_palette("pastel")

In [ ]:
# Distribution of the target variable
plt.figure(figsize=(12, 6))
sns.histplot(data['product_wg_ton'], kde=True, color='dodgerblue')
plt.title('Distribution of Target Variable')
plt.xlabel('Product Weight (in Tonne)')
plt.ylabel('Frequency')
plt.show()

Few insights from the plot:
1. Shape of the Distribution: The distribution appears to be roughly symmetrical, indicating a fairly even spread of product weights across the dataset.
2. Central Tendency: The peak of the distribution suggests that the most common product weights are around the central value.
3. Spread: The product weights vary significantly, ranging from around 2000 tons to over 55000 tons.
4. Density Plot: The KDE (Kernel Density Estimate) line provides a smooth estimate of the distribution, further highlighting the central tendency and spread.
5. Outliers: There are no significant outliers visible, which suggests that the data does not have extreme values that might skew the analysis.

### Data Preprocessing

In [ ]:
# Remove duplicates
data.drop_duplicates(inplace=True)

In [ ]:
# Dropping irrelevant columns
data.drop(['Ware_house_ID', 'WH_Manager_ID'], axis=1, inplace=True)

In [ ]:
# Handling missing values
# Distribution of the number of workers before imputation
plt.figure(figsize=(12, 6))
sns.histplot(data['workers_num'].dropna(), kde=True, color='dodgerblue', bins=30)
plt.title('Distribution of number of workers before Imputation')
plt.xlabel('Number of Workers')
plt.ylabel('Frequency')
plt.show()

Few insights from the plot:
1. Shape of the Distribution: The distribution is roughly symmetrical with a slight skew to the right, indicating that most warehouses have a moderate number of workers.
2. Central Tendency: The peak of the distribution suggests that the most common number of workers is around 30.
3. Spread: The number of workers varies significantly, ranging from about 10 to 60, with a few outliers extending up to 98.

In [ ]:
# Impute missing values for number of workers with the median
data['workers_num'].fillna(data['workers_num'].median(), inplace=True)

Justification: We use median imputation because it is more robust to outliers compared to mean.

In [ ]:
# Handling missing values for approved_wh_govt_certificate
print("Unique values in approved_wh_govt_certificate before imputation:")
print(data['approved_wh_govt_certificate'].unique())

print("Mode of approved_wh_govt_certificate:")
print(data['approved_wh_govt_certificate'].mode())

In [ ]:
# Impute missing values for approved_wh_govt_certificate with new category
data['approved_wh_govt_certificate'].fillna('NA', inplace=True)

Justification: The mode imputation would replace all missing values with the most frequent category. However, since approved_wh_govt_certificate is a categorical feature with specific meanings, filling with 'NA' allows us to retain the information that the certificate status is unknown rather than assuming it is the mode.

In [ ]:
# Select only numeric columns for correlation matrix calculation
numeric_data = data.select_dtypes(include=[np.number])

# Skewness and Kurtosis
skewness = numeric_data.skew()
kurtosis = numeric_data.kurtosis()

print("Skewness of the dataset:\n", skewness)
print("\nKurtosis of the dataset:\n", kurtosis)

Insights:
1. Transport issues and number of competitors show a positive skew, indicating that higher values are less frequent.
2. Flood impact and flood proof have high kurtosis values, indicating heavy tails or outliers in the distribution.
3. Electric supply and distance from the hub show negative skew, indicating that lower values are less frequent.

In [ ]:
# Detect outliers using IQR
Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1
outliers = numeric_data[(numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR))]

# Filter rows with any outliers
rows_with_outliers = data[outliers.any(axis=1)]

print("Rows with outliers:\n", rows_with_outliers)

In [ ]:
# Inspecting outliers
numeric_data1 = ['num_refill_req_l3m', 'transport_issue_l1y', 'Competitor_in_mkt',
       'retail_shop_num', 'distributor_num', 'flood_impacted', 'flood_proof',
       'electric_supply']

# Create box plots for all numeric columns
plt.figure(figsize=(12, 12))
for i, column in enumerate(numeric_data1, 1):
    plt.subplot(len(numeric_data1) // 2 + 1, 2, i)
    sns.boxplot(data=data[column], color='dodgerblue')
    plt.title(column)
    plt.tight_layout()
plt.show()

In [ ]:
# Inspecting outliers
numeric_data2 = ['dist_from_hub', 'workers_num', 'wh_est_year',
       'storage_issue_reported_l3m', 'temp_reg_mach', 'wh_breakdown_l3m',
       'govt_check_l3m', 'product_wg_ton']

# Create box plots for all numeric columns
plt.figure(figsize=(12, 12))
for i, column in enumerate(numeric_data2, 1):
    plt.subplot(len(numeric_data2) // 2 + 1, 2, i)
    sns.boxplot(data=data[column], color='dodgerblue')
    plt.title(column)
    plt.tight_layout()
plt.show()

In [ ]:
# Distribution of the numerical variables
for column in numeric_data.columns:
    plt.figure(figsize=(12, 6))
    sns.histplot(numeric_data[column], kde=True, color='dodgerblue')
    plt.title(f'Distribution of {column} Variable')
    plt.xlabel(f'{column}')
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
# Create pair plot
plt.figure(figsize=(12, 6))
sns.pairplot(numeric_data, palette='dodgerblue')
plt.show()

In [ ]:
# Creating copy for visualisation
datavis = data.copy()

### Exploratory Data Analysis

I. Warehouse Capacity:

In [ ]:
# Convert 'WH_capacity_size' to categorical labels for plotting purposes
datavis['WH_capacity_size'] = datavis['WH_capacity_size'].astype(pd.CategoricalDtype(categories=['Small', 'Mid', 'Large'], ordered=True))

# Calculate the counts of each warehouse capacity size
wh_cap_counts = datavis['WH_capacity_size'].value_counts().sort_index()
# Find the maximum count of warehouses
wh_cap_max_cat = wh_cap_counts.max()
# Create custom color palette based on the maximum count
wh_cap_palette = ['dodgerblue' if count == wh_cap_max_cat else 'lightskyblue' for count in wh_cap_counts]

# Plot the number of warehouses by capacity size
plt.figure(figsize=(12, 6))
sns.countplot(data=datavis, x='WH_capacity_size', hue='WH_capacity_size', palette=wh_cap_palette, legend=False)
plt.title('Number of Warehouses by Warehouse Capacity Size')
plt.xlabel('Warehouse Capacity Size')
plt.ylabel('Number of Warehouses')
plt.show()

Insights:
1. The 'Large' capacity size has the highest count among the three categories.

In [ ]:
# Calculate the counts of each warehouse capacity size
wh_cap_counts2 = datavis['zone'].value_counts().sort_index()
# Find the maximum count of warehouses
wh_cap_max_cat2 = wh_cap_counts2.max()
# Create custom color palette based on the maximum count
wh_cap_palette2 = ['dodgerblue' if count == wh_cap_max_cat2 else 'lightskyblue' for count in wh_cap_counts2]

# Plot the number of warehouses by capacity size
plt.figure(figsize=(12, 6))
sns.countplot(data=datavis, x='zone', hue='zone', palette=wh_cap_palette2, legend=False)
plt.title('Number of Warehouses by Zone')
plt.xlabel('Zone')
plt.ylabel('Number of Warehouses')
plt.show()

In [ ]:
# Calculate the total weight of product shipped by warehouse capacity size
wh_cap_prod_tot = datavis.groupby('WH_capacity_size', observed=False)['product_wg_ton'].sum().reset_index()

# Find the maximum total weight of product shipped
wh_cap_prod_max_cat = wh_cap_prod_tot['product_wg_ton'].max()

# Create custom color palette based on the maximum total weight of product shipped
wh_cap_prod_palette = ['dodgerblue' if value == wh_cap_prod_max_cat else 'lightskyblue' for value in wh_cap_prod_tot['product_wg_ton']]

# Plot the total weight of product shipped by warehouse capacity size
plt.figure(figsize=(12, 6))
plt.bar(wh_cap_prod_tot['WH_capacity_size'], wh_cap_prod_tot['product_wg_ton'], color=wh_cap_prod_palette)
plt.title('Total Weight of Product Shipped by Warehouse Capacity Size')
plt.xlabel('Warehouse Capacity Size')
plt.ylabel('Total Weight of Product Shipped (tons)')
plt.show()

Insights:
1. The 'Large' capacity warehouses ship the highest total weight of products compared to 'Mid' and 'Small' capacity warehouses.

In [ ]:
# Calculate the average weight of product shipped by warehouse capacity size
wh_cap_prod_avg = datavis.groupby('WH_capacity_size', observed=False)['product_wg_ton'].mean().reset_index()

# Find the maximum average weight
wh_cap_prod_max2 = wh_cap_prod_avg['product_wg_ton'].max()

# Create custom color palette based on the maximum average weight
wh_cap_prod_palette2 = ['dodgerblue' if value == wh_cap_prod_max2 else 'lightskyblue' for value in wh_cap_prod_avg['product_wg_ton']]

# Plot the average weight of product shipped by warehouse capacity size
plt.figure(figsize=(12, 6))
plt.bar(wh_cap_prod_avg['WH_capacity_size'], wh_cap_prod_avg['product_wg_ton'], color=wh_cap_prod_palette2)
plt.title('Average Weight of Product Shipped by Warehouse Capacity Size')
plt.xlabel('Warehouse Capacity Size')
plt.ylabel('Average Weight of Product Shipped (tons)')
plt.show()

Insights:
1. The 'Mid' capacity warehouses have the highest average weight of products shipped compared to 'Large' and 'Small' capacity warehouses.

In [ ]:
# Calculate the mean number of workers for each capacity size
wh_cap_worker_avg = datavis.groupby('WH_capacity_size', observed=False)['workers_num'].mean().reset_index()

# Find the maximum average number of workers
wh_cap_worker_max = wh_cap_worker_avg['workers_num'].max()

# Create custom color palette based on the maximum average number of workers
wh_cap_worker_palette = ['dodgerblue' if value == wh_cap_worker_max else 'lightskyblue' for value in wh_cap_worker_avg['workers_num']]

# Plot the mean number of workers by capacity size with customized colors
plt.figure(figsize=(12, 6))
plt.bar(wh_cap_worker_avg['WH_capacity_size'], wh_cap_worker_avg['workers_num'], color=wh_cap_worker_palette)
plt.title('Average Number of Workers and Warehouse Capacity Size')
plt.xlabel('Warehouse Capacity Size')
plt.ylabel('Average Number of Workers')
plt.show()

Insights:
1. The 'Small' capacity warehouses have the highest average number of workers compared to 'Mid' and 'Large' capacity warehouses.

II. Location and Accessibility:

In [ ]:
# Grouping by 'zone' and calculating the mean 'dist_from_hub'
wh_loc_dist_avg = datavis.groupby('zone')['dist_from_hub'].mean().reset_index()

# Find the maximum average distance from hub
wh_loc_dist_max_cat = wh_loc_dist_avg['dist_from_hub'].max()

# Create custom color palette based on the maximum average distance from hub
wh_loc_dist_palette = ['dodgerblue' if value == wh_loc_dist_max_cat else 'lightskyblue' for value in wh_loc_dist_avg['dist_from_hub']]

# Plotting with seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_loc_dist_avg, x='zone', y='dist_from_hub', hue='zone', palette=wh_loc_dist_palette, legend=False)
plt.title('Average Distance from Hub by Zone')
plt.xlabel('Zone')
plt.ylabel('Average Distance from Hub')
plt.grid(True)
plt.show()

In [ ]:
# Grouping by 'Location_type' and calculating the mean 'dist_from_hub'
wh_loc_loc_avg = datavis.groupby('Location_type')['dist_from_hub'].mean().reset_index()

# Find the maximum average distance from hub
wh_loc_loc_max_cat = wh_loc_loc_avg['dist_from_hub'].max()

# Create custom color palette based on the maximum average distance from hub
wh_loc_loc_palette = ['dodgerblue' if value == wh_loc_loc_max_cat else 'lightskyblue' for value in wh_loc_loc_avg['dist_from_hub']]

# Plotting with seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_loc_loc_avg, x='Location_type', y='dist_from_hub', hue='Location_type', palette=wh_loc_loc_palette, legend=False)
plt.title('Average Distance from Hub by Warehouse Location Type')
plt.xlabel('Warehouse Location Type')
plt.ylabel('Average Distance from Hub')
plt.show()

Insights:
1. Warehouses located in rural areas have a higher average distance from the hub compared to those in urban areas.

III. Warehouse Operations:

In [ ]:
# Filter out rows with known establishment years
wh_op_est_filter = datavis.dropna(subset=['wh_est_year'])

# Group by establishment year and calculate the mean of storage issues reported and warehouse breakdowns
wh_op_str_avg = wh_op_est_filter.groupby('wh_est_year')['storage_issue_reported_l3m'].mean().reset_index()
wh_op_break_avg = datavis.groupby('wh_est_year')['wh_breakdown_l3m'].mean().reset_index()

# Plotting with seaborn
plt.figure(figsize=(12, 6))
sns.lineplot(data=wh_op_str_avg, x='wh_est_year', y='storage_issue_reported_l3m', marker='o', color='dodgerblue', label='Storage Issues (last 3 months)')
sns.lineplot(data=wh_op_break_avg, x='wh_est_year', y='wh_breakdown_l3m', marker='o', color='gold', label='Warehouse Breakdowns (last 3 months)')
plt.title('Trends in Storage Issues and Warehouse Breakdowns Over Time')
plt.xlabel('Year of Establishment')
plt.ylabel('Number of Incidents')
plt.legend()
plt.show()

In [ ]:
# Calculate the average product weight shipped grouped by the number of workers
wh_op_worker_wt_avg = data.groupby('workers_num')['product_wg_ton'].mean().reset_index()

# Plotting with seaborn
plt.figure(figsize=(12, 6))
sns.lineplot(data=wh_op_worker_wt_avg, x='workers_num', y='product_wg_ton', marker='o', color='dodgerblue')
plt.title('Influence of Number of Workers on Product Weight Shipped')
plt.xlabel('Number of Workers')
plt.ylabel('Average Product Weight Shipped (tons)')
plt.show()

Insights:
1. Initially, as the number of workers increases, there seems to be an increase in the average weight shipped. After reaching a peak, the trend shows variability, indicating that other factors might also influence the product weight shipped.


In [ ]:
# Calculate total values for each issue by warehouse capacity size
wh_op_issues_tot = datavis.groupby('WH_capacity_size', observed=False)[['storage_issue_reported_l3m', 'wh_breakdown_l3m', 'num_refill_req_l3m']].sum().reset_index()

# Find the maximum value for each issue
wh_op_strissue_max_cat = wh_op_issues_tot['storage_issue_reported_l3m'].max()
wh_op_breakissue_max_cat = wh_op_issues_tot['wh_breakdown_l3m'].max()
wh_op_refillissue_max_cat = wh_op_issues_tot['num_refill_req_l3m'].max()

# Create color schemes for each issue
wh_op_strissue_palette = ['dodgerblue' if value == wh_op_strissue_max_cat else 'lightskyblue' for value in wh_op_issues_tot['storage_issue_reported_l3m']]
wh_op_breakissue_palette = ['gold' if value == wh_op_breakissue_max_cat else 'khaki' for value in wh_op_issues_tot['wh_breakdown_l3m']]
wh_op_refillissue_palette = ['salmon' if value == wh_op_refillissue_max_cat else 'lightsalmon' for value in wh_op_issues_tot['num_refill_req_l3m']]

# Plot each bar group separately to apply different colors
fig, ax = plt.subplots(figsize=(12, 6))
bar_width = 0.25
bar_positions = np.arange(len(wh_op_issues_tot))

# Plot each bar group separately to apply different colors
ax.bar(bar_positions, wh_op_issues_tot['storage_issue_reported_l3m'], width=bar_width, color=wh_op_strissue_palette, label='Storage Issues')
ax.bar(bar_positions + bar_width, wh_op_issues_tot['wh_breakdown_l3m'], width=bar_width, color=wh_op_breakissue_palette, label='Breakdowns')
ax.bar(bar_positions + 2 * bar_width, wh_op_issues_tot['num_refill_req_l3m'], width=bar_width, color=wh_op_refillissue_palette, label='Refill Required')

# Set labels and title
ax.set_xticks(bar_positions + bar_width)
ax.set_xticklabels(wh_op_issues_tot['WH_capacity_size'])
ax.set_title('Total Operational Issues by Warehouse Capacity Size')
ax.set_xlabel('Warehouse Capacity Size')
ax.set_ylabel('Total Number of Issues (last 3 months)')
ax.legend()
plt.show()

Insights:
1. Storage Issues: The 'Large' capacity warehouses have the highest total number of storage issues reported in the last three months.
2. Breakdowns: Similarly, the 'Large' capacity warehouses also have the highest total number of warehouse breakdowns.
3. Refill Requirements: The 'Large' capacity warehouses again show the highest total number of refills required, indicating more frequent refill needs.

In [ ]:
# Calculate total values for each issue by warehouse capacity size
wh_op_issues_avg = datavis.groupby('WH_capacity_size', observed=False)[['storage_issue_reported_l3m', 'wh_breakdown_l3m', 'num_refill_req_l3m']].mean().reset_index()

# Find the maximum value for each issue
wh_op_strissue_max_cat2 = wh_op_issues_avg['storage_issue_reported_l3m'].max()
wh_op_breakissue_max_cat2 = wh_op_issues_avg['wh_breakdown_l3m'].max()
wh_op_refillissue_max_cat2 = wh_op_issues_avg['num_refill_req_l3m'].max()

# Plot each bar group separately to apply different colors
fig, ax = plt.subplots(figsize=(12, 6))
bar_width = 0.25
bar_positions = np.arange(len(wh_op_issues_avg))

# Plot each bar group separately to apply different colors
ax.bar(bar_positions, wh_op_issues_avg['storage_issue_reported_l3m'], width=bar_width, color=wh_op_strissue_palette, label='Storage Issues')
ax.bar(bar_positions + bar_width, wh_op_issues_avg['wh_breakdown_l3m'], width=bar_width, color=wh_op_breakissue_palette, label='Breakdowns')
ax.bar(bar_positions + 2 * bar_width, wh_op_issues_avg['num_refill_req_l3m'], width=bar_width, color=wh_op_refillissue_palette, label='Refill Required')

# Set labels and title
ax.set_xticks(bar_positions + bar_width)
ax.set_xticklabels(wh_op_issues_avg['WH_capacity_size'])
ax.set_title('Total Operational Issues by Warehouse Capacity Size')
ax.set_xlabel('Warehouse Capacity Size')
ax.set_ylabel('Total Number of Issues (last 3 months)')
ax.legend()
plt.show()

Iinsights:
1. Storage Issues: The 'Large' capacity warehouses have the highest average number of storage issues reported in the last three months.
2. Breakdowns: Similarly, the 'Large' capacity warehouses also have the highest average number of warehouse breakdowns.
3. Refill Requirements: The 'Large' capacity warehouses again show the highest average number of refills required, indicating more frequent refill needs.

In [ ]:
# Calculate the average storage issues reported by location type
wh_op_loc_str_avg = data.groupby('Location_type')['storage_issue_reported_l3m'].mean().reset_index()

# Determine the maximum value for coloring
wh_op_loc_str_max_cat = wh_op_loc_str_avg['storage_issue_reported_l3m'].max()

# Create a color palette based on the maximum value
wh_op_loc_str_palette = ['dodgerblue' if value == wh_op_loc_str_max_cat else 'lightskyblue' for value in wh_op_loc_str_avg['storage_issue_reported_l3m']]

# Plot with Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_op_loc_str_avg, x='Location_type', y='storage_issue_reported_l3m', hue='Location_type', palette=wh_op_loc_str_palette, legend=False)
plt.title('Storage Issues Reported by Location Type')
plt.xlabel('Location Type')
plt.ylabel('Average Storage Issues Reported')
plt.show()

Insights:
1. Warehouses located in urban areas report a higher average number of storage issues compared to those in rural areas.

In [ ]:
# Calculate the average storage issues reported by zone
wh_op_zone_str_avg = data.groupby('zone')['storage_issue_reported_l3m'].mean().reset_index()

# Determine the maximum value for coloring
wh_op_zone_str_max_cat = wh_op_zone_str_avg['storage_issue_reported_l3m'].max()

# Create a color palette based on the maximum value
wh_op_zone_str_palette = ['dodgerblue' if value == wh_op_zone_str_max_cat else 'lightskyblue' for value in wh_op_zone_str_avg['storage_issue_reported_l3m']]

# Plot with Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_op_zone_str_avg, x='zone', y='storage_issue_reported_l3m', hue='zone', palette=wh_op_zone_str_palette, legend=False)
plt.title('Storage Issues Reported by Zone')
plt.xlabel('Zone')
plt.ylabel('Average Storage Issues Reported')
plt.show()

IV. Logistics and Transportation:

In [ ]:
# Calculate the average transport issues reported by zone
wh_logi_zone_avg = datavis.groupby('zone')['transport_issue_l1y'].mean().reset_index()

# Determine the maximum value for coloring
wh_logi_zone_max_cat = wh_logi_zone_avg['transport_issue_l1y'].max()

# Create color palette based on the maximum value
wh_logi_zone_palette = ['dodgerblue' if value == wh_logi_zone_max_cat else 'lightskyblue' for value in wh_logi_zone_avg['transport_issue_l1y']]

# Plot with Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_logi_zone_avg, x='zone', y='transport_issue_l1y', hue='zone', palette=wh_logi_zone_palette, legend=False)
plt.title('Transport Issues (last year) by Zone')
plt.xlabel('Zone')
plt.ylabel('Average Transport Issues (last year)')
plt.show()

In [ ]:
# Calculate the average transport issues reported by Location_type
wh_logi_loc_avg = datavis.groupby('Location_type')['transport_issue_l1y'].mean().reset_index()

# Determine the maximum value for coloring
wh_logi_loc_max_cat = wh_logi_loc_avg['transport_issue_l1y'].max()

# Create color palette based on the maximum value
wh_logi_loc_palette = ['dodgerblue' if value == wh_logi_loc_max_cat else 'lightskyblue' for value in wh_logi_loc_avg['transport_issue_l1y']]

# Plot with Seaborn
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_logi_loc_avg, x='Location_type', y='transport_issue_l1y', hue='Location_type', palette=wh_logi_loc_palette, legend=False)
plt.title('Transport Issues (last year) by Location Type')
plt.xlabel('Location Type')
plt.ylabel('Average Transport Issues (last year)')
plt.show()

Insights:
1. The chart shows that warehouses in urban areas experience a higher average number of transport issues compared to those in rural areas.

In [ ]:
# Bar plot for average transport issues by distance from hub
wh_logi_dist_avg = datavis.groupby('transport_issue_l1y')['dist_from_hub'].mean().reset_index()

# Define the conditional coloring logic
wh_logi_dist_max_cat = wh_logi_dist_avg['dist_from_hub'].max()
wh_logi_dist_palette = ['dodgerblue' if value == wh_logi_dist_max_cat else 'lightskyblue' for value in wh_logi_dist_avg['dist_from_hub']]

# Plotting the bar chart
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_logi_dist_avg, y='dist_from_hub', x='transport_issue_l1y', hue='transport_issue_l1y', palette=wh_logi_dist_palette, legend=False)
plt.title('Average Distance from Hub by Transport Issues (last year)')
plt.ylabel('Average Distance from Hub')
plt.xlabel('Transport Issues (last year)')
plt.show()

In [ ]:
# Bar plot for average transport issues by product weight
wh_logi_prod_avg = datavis.groupby('transport_issue_l1y')['product_wg_ton'].mean().reset_index()

# Define the conditional coloring logic
wh_logi_prod_max_cat = wh_logi_prod_avg['product_wg_ton'].max()
wh_logi_prod_palette = ['dodgerblue' if value == wh_logi_prod_max_cat else 'lightskyblue' for value in wh_logi_prod_avg['product_wg_ton']]

# Plotting the bar chart
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_logi_prod_avg, y='product_wg_ton', x='transport_issue_l1y', hue='transport_issue_l1y', palette=wh_logi_prod_palette, legend=False)
plt.title('Average Product Weight Shipped by Transport Issues (last year)')
plt.ylabel('Average Product Weight Shipped (tons)')
plt.xlabel('Transport Issues (last year)')
plt.show()

V. Competition and Market Factors:

In [ ]:
# Define the bin edges
wh_mar_bin_edges = [0, 3, 6, 9, 12]

# Define the bin labels
wh_mar_bin_labels = ['0-3', '4-6', '7-9', '10-12']

# Create a new column in the DataFrame to represent the bins
datavis['Competitor_in_mkt_bins'] = pd.cut(datavis['Competitor_in_mkt'], bins=wh_mar_bin_edges, labels=wh_mar_bin_labels, right=False)

# Bar plot for Competitors in Market vs. Number of Retail Shops, with hue representing the bins
plt.figure(figsize=(12, 6))
sns.barplot(data=datavis, x='retail_shop_num', y='zone', hue='Competitor_in_mkt_bins', estimator=sum, palette="Blues")
plt.title('Total Number of Retail Shops by Zone and Number of Competitors in Market')
plt.xlabel('Total Number of Retail Shops')
plt.ylabel('Zone')
plt.legend(title='Number of Competitors in Market')
plt.show()

In [ ]:
# Bar plot for Competitors in Market vs. Number of Retail Shops, with hue representing the bins
plt.figure(figsize=(12, 6))
sns.barplot(data=datavis, x='retail_shop_num', y='Location_type', hue='Competitor_in_mkt_bins', estimator=sum, palette="Blues")
plt.title('Total Number of Retail Shops by Warehouse Location Type and Number of Competitors in Market')
plt.xlabel('Total Number of Retail Shops')
plt.ylabel('Warehouse Location Type')
plt.legend(title='Number of Competitors in Market')
plt.show()

In [ ]:
# Competitors' Influence on Warehouse Performance
wh_mar_prod_tot = datavis.groupby('Competitor_in_mkt')['product_wg_ton'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=wh_mar_prod_tot, x='Competitor_in_mkt', y='product_wg_ton', marker='o', color='dodgerblue')
plt.title('Product Weight Shipped by Number of Competitors')
plt.xlabel('Number of Competitors')
plt.ylabel('Product Weight Shipped (tons)')
plt.show()

In [ ]:
# Competitors' Influence on Warehouse Performance
plt.figure(figsize=(12, 6))
sns.lineplot(data=datavis, x='Competitor_in_mkt', y='product_wg_ton', marker='o', color='dodgerblue')
plt.title('Average Product Weight Shipped by Number of Competitors')
plt.xlabel('Number of Competitors')
plt.ylabel('Average Product Weight Shipped (tons)')
plt.show()

VI. Compliance and Regulatory Factors

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=datavis, x='govt_check_l3m', y='zone', hue='approved_wh_govt_certificate', estimator='mean', palette='Blues')
plt.title('Total Government Checks by Zone and Approved Govt Certificate')
plt.xlabel('Total Government Checks (last 3 months)')
plt.ylabel('Zone')
plt.legend(title='Approved Govt Certificate')
plt.show()

In [ ]:
# Create bins for government checks
wh_comp_bin_edges = [0, 5, 10, 15, 20, 25, np.inf]
wh_comp_bin_labels = ['0-5', '6-10', '11-15', '16-20', '21-25', '>25']

datavis['govt_check_bins'] = pd.cut(datavis['govt_check_l3m'], bins=wh_comp_bin_edges, labels=wh_comp_bin_labels, right=False)

# Group by government check bins and calculate mean storage issues
wh_comp_str_avg = datavis.groupby('govt_check_bins', observed=False)['storage_issue_reported_l3m'].mean().reset_index()

# Determine the maximum value for coloring
wh_comp_str_max_cat = wh_comp_str_avg['storage_issue_reported_l3m'].max()

# Create color palette based on the maximum value
comp_str_palette = ['dodgerblue' if value == wh_comp_str_max_cat else 'lightskyblue' for value in wh_comp_str_avg['storage_issue_reported_l3m']]

# Plot the data
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_comp_str_avg, x='govt_check_bins', y='storage_issue_reported_l3m', hue='govt_check_bins', palette=comp_str_palette, legend=False)
plt.title('Average Storage Issues by Government Checks')
plt.xlabel('Government Checks (last 3 months)')
plt.ylabel('Average Storage Issues Reported (last 3 months)')
plt.grid(True)
plt.show()

In [ ]:
# Group by government check bins and calculate mean storage issues
wh_comp_break_avg = datavis.groupby('govt_check_bins', observed=False)['wh_breakdown_l3m'].mean().reset_index()

# Determine the maximum value for coloring
wh_comp_break_max_cat = wh_comp_break_avg['wh_breakdown_l3m'].max()

# Create color palette based on the maximum value
comp_break_palette = ['dodgerblue' if value == wh_comp_break_max_cat else 'lightskyblue' for value in wh_comp_break_avg['wh_breakdown_l3m']]

# Plot the data
plt.figure(figsize=(12, 6))
sns.barplot(data=wh_comp_break_avg, x='govt_check_bins', y='wh_breakdown_l3m', hue='govt_check_bins', palette=comp_break_palette, legend=False)
plt.title('Average Warehouse Breakdowns by Government Checks')
plt.xlabel('Government Checks (last 3 months)')
plt.ylabel('Average Warehouse Breakdowns Reported (last 3 months)')
plt.grid(True)
plt.show()

VII. Infrastructure and Facilities

In [ ]:
# Calculate total values for each issue by warehouse capacity size
wh_inf_issues_avg = datavis.groupby('flood_impacted', observed=False)[['storage_issue_reported_l3m', 'wh_breakdown_l3m']].mean().reset_index()

# Find the maximum value for each issue
wh_inf_strissue_max_cat = wh_inf_issues_avg['storage_issue_reported_l3m'].max()
wh_inf_breakissue_max_cat = wh_inf_issues_avg['wh_breakdown_l3m'].max()

# Create color schemes for each issue
wh_inf_strissue_palette = ['dodgerblue' if value == wh_inf_strissue_max_cat else 'lightskyblue' for value in wh_inf_issues_avg['storage_issue_reported_l3m']]
wh_inf_breakissue_palette = ['gold' if value == wh_inf_breakissue_max_cat else 'khaki' for value in wh_inf_issues_avg['wh_breakdown_l3m']]

# Plot each bar group separately to apply different colors
fig, ax = plt.subplots(figsize=(12, 6))
bar_width = 0.4
bar_positions = np.arange(len(wh_inf_issues_avg))

# Plot each bar group separately to apply different colors
ax.bar(bar_positions, wh_inf_issues_avg['storage_issue_reported_l3m'], width=bar_width, color=wh_inf_strissue_palette, label='Storage Issues')
ax.bar(bar_positions + bar_width, wh_inf_issues_avg['wh_breakdown_l3m'], width=bar_width, color=wh_inf_breakissue_palette, label='Breakdowns')

# Set labels and title
ax.set_xticks(bar_positions + bar_width/2)
ax.set_xticklabels(['No', 'Yes'])
ax.set_title('Total Operational Issues by Flood Impact')
ax.set_xlabel('Flood Impact')
ax.set_ylabel('Average Number of Issues (last 3 months)')
ax.legend()
plt.show()

In [ ]:
# Calculate total values for each issue by warehouse capacity size
wh_inf_issues_avg2 = datavis.groupby('electric_supply', observed=False)[['storage_issue_reported_l3m', 'wh_breakdown_l3m']].mean().reset_index()

# Find the maximum value for each issue
wh_inf_strissue_max_cat2 = wh_inf_issues_avg2['storage_issue_reported_l3m'].max()
wh_inf_breakissue_max_cat2 = wh_inf_issues_avg2['wh_breakdown_l3m'].max()

# Create color schemes for each issue
wh_inf_strissue_palette2 = ['dodgerblue' if value == wh_inf_strissue_max_cat2 else 'lightskyblue' for value in wh_inf_issues_avg2['storage_issue_reported_l3m']]
wh_inf_breakissue_palette2 = ['gold' if value == wh_inf_breakissue_max_cat2 else 'khaki' for value in wh_inf_issues_avg2['wh_breakdown_l3m']]

# Plot each bar group separately to apply different colors
fig, ax = plt.subplots(figsize=(12, 6))
bar_width = 0.4
bar_positions = np.arange(len(wh_inf_issues_avg2))

# Plot each bar group separately to apply different colors
ax.bar(bar_positions, wh_inf_issues_avg2['storage_issue_reported_l3m'], width=bar_width, color=wh_inf_strissue_palette2, label='Storage Issues')
ax.bar(bar_positions + bar_width, wh_inf_issues_avg2['wh_breakdown_l3m'], width=bar_width, color=wh_inf_breakissue_palette2, label='Breakdowns')

# Set labels and title
ax.set_xticks(bar_positions + bar_width/2)
ax.set_xticklabels(['No', 'Yes'])
ax.set_title('Total Operational Issues by Availability of Electricity')
ax.set_xlabel('Availability of Electricity')
ax.set_ylabel('Average Number of Issues (last 3 months)')
ax.legend()
plt.show()

In [ ]:
# Group by government check bins and calculate mean storage issues
wh_inf_prod_avg = datavis.groupby('temp_reg_mach', observed=False)['product_wg_ton'].mean().reset_index()

# Determine the maximum value for coloring
wh_inf_prod_max_cat = wh_inf_prod_avg['product_wg_ton'].max()

# Create color palette based on the maximum value
wh_inf_prod_palette = ['dodgerblue' if value == wh_inf_prod_max_cat else 'lightskyblue' for value in wh_inf_prod_avg['product_wg_ton']]

# Plot the data
plt.figure(figsize=(12, 6))
ax = sns.barplot(data=wh_inf_prod_avg, x='temp_reg_mach', y='product_wg_ton', hue='temp_reg_mach', palette=wh_inf_prod_palette, legend=False)
plt.title('Average Product Weight Shipped by Temperature Regulation Machinery')
plt.xlabel('Temperature Regulation Machinery')
plt.ylabel('Average Product Weight Shipped (tons)')
ax.set_xticks(range(len(wh_inf_prod_avg)))
ax.set_xticklabels(['No', 'Yes'])
plt.show()

VIII. Historical Trend

In [ ]:
# Historical trend analysis (for warehouses with known establishment years)
wh_est_year_trend = datavis.dropna(subset=['wh_est_year']).groupby('wh_est_year')['product_wg_ton'].sum()
plt.figure(figsize=(12, 6))
sns.lineplot(data=wh_est_year_trend, marker='o', color='dodgerblue')
plt.title('Historical Trend of Product Weight Shipped')
plt.xlabel('Year of Establishment')
plt.ylabel('Total Product Weight (tons)')
plt.show()

In [ ]:
# Historical trend analysis (for warehouses with known establishment years)
wh_est_year_trend2 = datavis.dropna(subset=['wh_est_year']).groupby('wh_est_year')['product_wg_ton'].mean()
plt.figure(figsize=(12, 6))
sns.lineplot(data=wh_est_year_trend2, marker='o', color='dodgerblue')
plt.title('Historical Trend of Product Weight Shipped')
plt.xlabel('Year of Establishment')
plt.ylabel('Average Product Weight (tons)')
plt.show()

In [ ]:
# Correlation Matrix
correlation_matrix = numeric_data.corr()

plt.figure(figsize=(16, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='Blues', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

### Feature Engineering & Baseline Modelling

In [ ]:
# Create indicator variable for wh_est_year missingness
data['wh_est_year_missing'] = data['wh_est_year'].isnull().astype(int)

In [ ]:
# Feature Importance Analysis before proceeding with missing value handling and model building.
# Use a placeholder value that is clearly outside the valid range for 'wh_est_year'
year_placeholder = -1
data['wh_est_year'] = data['wh_est_year'].fillna(year_placeholder)

features = data.drop(columns=['product_wg_ton'])
target = data['product_wg_ton']
features_encoded = pd.get_dummies(features, drop_first=True)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(features_encoded, target)

feature_importances = rf.feature_importances_
feature_names = features_encoded.columns

feature_importance_data = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_data = feature_importance_data.sort_values(by='Importance', ascending=False)
print(feature_importance_data)

In [ ]:
# Visualize most important features
plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_data.head(15), color='dodgerblue', legend=False)
plt.title('Top 15 Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
# Creating base model v1 (Single Variable Linear Regression)
# Select the most important feature and the target variable
base_X = data[['storage_issue_reported_l3m']]
base_y = data['product_wg_ton']

# Split the data into training and testing sets
base_X_train, base_X_test, base_y_train, base_y_test = train_test_split(base_X, base_y, test_size=0.2, random_state=42)

# Initialize the scaler
base_sc = StandardScaler()

# Fit the scaler on the training data and transform the training data
base_X_train_scaled = base_sc.fit_transform(base_X_train)

# Transform the test data using the already fitted scaler
base_X_test_scaled = base_sc.transform(base_X_test)
 
# Create and fit the model
base_model = LinearRegression()
base_model.fit(base_X_train_scaled, base_y_train)

# Make predictions
base_y_pred = base_model.predict(base_X_test_scaled)

# Evaluate the model
base_mae = mean_absolute_error(base_y_test, base_y_pred)
base_mse = mean_squared_error(base_y_test, base_y_pred)
base_r2 = r2_score(base_y_test, base_y_pred)

print(f'Mean Absolute Error: {base_mae}')
print(f'Mean Squared Error: {base_mse}')
print(f'R-squared: {base_r2}')

In [ ]:
# Coefficients and intercept from the model
beta = base_model.coef_[0]
beta_0 = base_model.intercept_

print("Model Coefficients:", beta)
print("Model Intercept:", beta_0)

# Mean and standard deviation used for scaling
mu = base_sc.mean_
sigma = base_sc.scale_

# Calculate new coefficients and intercept
alpha_0 = beta_0 - np.sum(beta * (mu / sigma))
alpha = beta / sigma

print("Reverse Scaled Intercept (alpha_0):", alpha_0)
print("Reverse Scaled Coefficients (alpha):", alpha)

# Final equation
equation = f"y = {alpha_0} + ({alpha[0]}) * x1"
print("Base Equation:", equation)

In [ ]:
# Reverse scaled coefficients and intercept
alpha_0 = 689.4218640991749
alpha_1 = 1249.9555675709278

# Generate a range of values for x1
x1_range = np.linspace(0, 50)

# Calculate corresponding y values
y = alpha_0 + alpha_1 * x1_range

# Plot the equation
plt.figure(figsize=(12, 6))
plt.plot(x1_range, y, label=f'y = {alpha_0:.2f} + {alpha_1:.2f} * x1', color='dodgerblue')
plt.xlabel('Predictor')
plt.ylabel('Target')
plt.title('Base Linear Regression Line')
plt.legend()
plt.show()

In [ ]:
# Plot the results

# Plot 1: Actual vs. Predicted
plt.figure(figsize=(10, 6))
plt.scatter(base_y_test, base_y_pred, color='dodgerblue', label='Actual vs Predicted')
plt.plot([base_y_test.min(), base_y_test.max()], [base_y_test.min(), base_y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Linear Regression: Actual vs. Predicted')
plt.legend()
plt.show()

# Plot 2: Feature vs. Actual and Predicted
plt.figure(figsize=(10, 6))
plt.scatter(base_X_test, base_y_test, color='dodgerblue', label='Actual')
plt.scatter(base_X_test, base_y_pred, color='gold', label='Predicted')
plt.xlabel('Storage Issues Reported (Last 3 Months)')
plt.ylabel('Product Weight (ton)')
plt.title('Linear Regression: Feature vs. Actual and Predicted')
plt.legend()
plt.show()


In [ ]:
# Creating base model v2 (Linear Regression)
# Select the top 5 features and the target variable
base2_data_encoded = pd.get_dummies(data, drop_first=True)
base2_selected_features = ['storage_issue_reported_l3m', 'approved_wh_govt_certificate_B', 
                           'approved_wh_govt_certificate_B+']

base2_X = base2_data_encoded[base2_selected_features]
base2_y = data['product_wg_ton']

# Split the data into training and testing sets
base2_X_train, base2_X_test, base2_y_train, base2_y_test = train_test_split(base2_X, base2_y, test_size=0.2, random_state=42)

# Initialize the scaler
base2_sc = StandardScaler()

# Fit the scaler on the training data and transform the training data
base2_X_train_scaled = base2_sc.fit_transform(base2_X_train)

# Transform the test data using the already fitted scaler
base2_X_test_scaled = base2_sc.transform(base2_X_test)

# Create and fit the model
base2_model = LinearRegression()
base2_model.fit(base2_X_train_scaled, base2_y_train)

# Make predictions
base2_y_pred = base2_model.predict(base2_X_test_scaled)

# Evaluate the model
base2_mae = mean_absolute_error(base2_y_test, base2_y_pred)
base2_mse = mean_squared_error(base2_y_test, base2_y_pred)
base2_r2 = r2_score(base2_y_test, base2_y_pred)

print(f'Mean Absolute Error: {base2_mae}')
print(f'Mean Squared Error: {base2_mse}')
print(f'R-squared: {base2_r2}')


In [ ]:
# Plot the results

# Plot 1: Actual vs. Predicted
plt.figure(figsize=(12, 6))
plt.scatter(base2_y_test, base2_y_pred, color='dodgerblue', label='Actual vs Predicted')
plt.plot([base2_y_test.min(), base2_y_test.max()], [base2_y_test.min(), base2_y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Linear Regression: Actual vs. Predicted')
plt.legend()
plt.show()

# Plot 2: Feature vs. Actual and Predicted
# Assuming 'storage_issue_reported_l3m' is the first feature in base2_X_test
feature_index = 0
feature_name = base2_selected_features[feature_index]

plt.figure(figsize=(10, 6))
plt.scatter(base2_X_test.iloc[:, feature_index], base2_y_test, color='dodgerblue', label='Actual')
plt.scatter(base2_X_test.iloc[:, feature_index], base2_y_pred, color='gold', label='Predicted')
plt.xlabel(feature_name)
plt.ylabel('Product Weight (ton)')
plt.title('Linear Regression: Feature vs. Actual and Predicted')
plt.legend()
plt.show()


In [ ]:
# Dropping column with 48% missing data
df = data.drop(['wh_est_year', 'wh_est_year_missing'], axis=1)

In [ ]:
# Saving the dataframe as a CSV file
df.to_csv('processed_data.csv', index=False)
print("Dataframe saved as 'processed_data.csv'")

In [ ]:
# Features and target variable
X = df.drop(columns=['product_wg_ton'])
y = df['product_wg_ton']

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and fit the RFE model
model = LinearRegression()
rfe = RFE(model, n_features_to_select=11)
rfe = rfe.fit(X_train_scaled, y_train)

# Select the most important features
selected_features = X_encoded.columns[rfe.support_]
print("Selected features:", selected_features)

# Fit the model with the selected features
X_train_selected = X_train_scaled[:, rfe.support_]
X_test_selected = X_test_scaled[:, rfe.support_]
model.fit(X_train_selected, y_train)

# Make predictions
y_pred = model.predict(X_test_selected)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='dodgerblue', label='Actual vs Predicted')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('RFE Linear Regression')
plt.legend()
plt.show()

In [ ]:
# Identify column indices for the selected features
selected_features = ['storage_issue_reported_l3m', 'approved_wh_govt_certificate_B', 'approved_wh_govt_certificate_B+']
selected_indices = [X_train.columns.get_loc(feature) for feature in selected_features]

# Prepare the data with selected features
X_train_selected = X_train_scaled[:, selected_indices]
X_test_selected = X_test_scaled[:, selected_indices]

# Function to evaluate model using cross-validation
def evaluate_model(model, X, y):
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    mae_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_absolute_error')
    mse_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
    r2_scores = cross_val_score(model, X, y, cv=kf, scoring='r2')
    
    return {
        'MAE': -mae_scores.mean(),
        'MSE': -mse_scores.mean(),
        'R2': r2_scores.mean()
    }

# Models to evaluate
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42)
}

# Evaluate models
results = {}
for name, model in models.items():
    results[name] = evaluate_model(model, X_train_selected, y_train)

# Display results
for model, metrics in results.items():
    print(f"{model} - MAE: {metrics['MAE']}, MSE: {metrics['MSE']}, R2: {metrics['R2']}")

# Plot results
metrics_names = ['MAE', 'MSE', 'R2']
n_models = len(models)
x = np.arange(n_models)

fig, axs = plt.subplots(1, 3, figsize=(12, 6))

for i, metric in enumerate(metrics_names):
    values = [results[model][metric] for model in models]
    axs[i].bar(x, values)
    axs[i].set_xticks(x)
    axs[i].set_xticklabels(models.keys())
    axs[i].set_title(metric)
    axs[i].set_xlabel('Model')
    axs[i].set_ylabel(metric)

plt.tight_layout()
plt.show()

In [ ]:
# Building final model
# Perform one-hot encoding
lr_df_encoded = pd.get_dummies(df, drop_first=True)
lr_selected_features = ['storage_issue_reported_l3m', 'approved_wh_govt_certificate_B', 
                        'approved_wh_govt_certificate_B+']

lr_X = lr_df_encoded[lr_selected_features]
lr_y = df['product_wg_ton']

# Split the data into training and testing sets
lr_X_train, lr_X_test, lr_y_train, lr_y_test = train_test_split(lr_X, lr_y, test_size=0.2, random_state=42)

# Initialize the scaler
lr_sc = StandardScaler()

# Fit the scaler on the training data and transform the training data
lr_X_train_scaled = lr_sc.fit_transform(lr_X_train)

# Transform the test data using the already fitted scaler
lr_X_test_scaled = lr_sc.transform(lr_X_test)

# Hyperparameter tuning and cross-validation
param_grid = {'fit_intercept': [True, False]}
grid_search = GridSearchCV(LinearRegression(), param_grid, cv=10, scoring='r2')
grid_search.fit(lr_X_train_scaled, lr_y_train)

best_model = grid_search.best_estimator_

# Print the best parameters
print("Best Parameters:")
print(grid_search.best_params_)

# Make predictions using the best model
lr_y_pred = best_model.predict(lr_X_test_scaled)

# Evaluate the model
lr_mae = mean_absolute_error(lr_y_test, lr_y_pred)
lr_mse = mean_squared_error(lr_y_test, lr_y_pred)
lr_r2 = r2_score(lr_y_test, lr_y_pred)

print(f'Mean Absolute Error: {lr_mae}')
print(f'Mean Squared Error: {lr_mse}')
print(f'R-squared: {lr_r2}')

# Plot the results
plt.figure(figsize=(12, 6))
plt.scatter(lr_y_test, lr_y_pred, color='dodgerblue')
plt.plot([lr_y_test.min(), lr_y_test.max()], [lr_y_test.min(), lr_y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.show()

In [ ]:
# Calculate residuals
residuals = lr_y_test - lr_y_pred

# Plot residuals
plt.figure(figsize=(12, 6))
plt.scatter(lr_y_pred, residuals, color='dodgerblue')
plt.hlines(y=0, xmin=lr_y_pred.min(), xmax=lr_y_pred.max(), colors='red', linestyles='dashed')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Predicted Values')
plt.show()

In [ ]:
# Mean and scale (standard deviation) from the scaler
best_mu = lr_sc.mean_
best_sigma = lr_sc.scale_

# Extract coefficients and intercept from the best model
best_coef_1 = best_model.coef_[0]
best_coef_2 = best_model.coef_[1]
best_coef_3 = best_model.coef_[2]
best_intercept = best_model.intercept_

print("Model Coefficient 1:", best_coef_1)
print("Model Coefficient 2:", best_coef_1)
print("Model Coefficient 3:", best_coef_1)
print("Model Intercept:", best_intercept)

# Calculate new coefficients and intercept considering the scaling
best_alpha_0 = best_intercept - (best_coef_1 * best_mu[0] / best_sigma[0]) - (best_coef_2 * best_mu[1] / best_sigma[1]) - (best_coef_3 * best_mu[2] / best_sigma[2])
best_alpha_1 = best_coef_1 / best_sigma[0]
best_alpha_2 = best_coef_2 / best_sigma[1]
best_alpha_3 = best_coef_3 / best_sigma[2]

print("Reverse Scaled Intercept (alpha_0):", best_alpha_0)
print("Reverse Scaled Coefficient 1 (alpha_1):", best_alpha_1)
print("Reverse Scaled Coefficient 2 (alpha_2):", best_alpha_2)
print("Reverse Scaled Coefficient 3 (alpha_3):", best_alpha_3)

# Final equation
best_equation = f"y = {best_alpha_0} + ({best_alpha_1}) * x1 + ({best_alpha_2}) * x2 + ({best_alpha_3}) * x3"
print("Final Equation:", best_equation)